Primero instalamos lo necesario para poder ejecutar pyOpenCl en el entorno

In [1]:
%%capture
!sudo apt -y update
!sudo apt install -y nvidia-cuda-toolkit
!sudo apt install -y pocl-opencl-icd
!pip install pyopencl
!pip install siphash24

Ahora importaremos las librerías necesarias a lo largo de la tarea

In [2]:
import numpy as np
import tensorflow_datasets as tfds
from matplotlib import pylab as plt
import pyopencl as cl
import pyopencl.array as cl_array
import time

Descargamos la base de datos, y vemos cuantos datos tenemos para verificar que se haya descargado correctamente

In [3]:
(mnist_data_train, mnist_data_test), data_info = tfds.load("mnist", split=["train", "test"], with_info=True, download=True)
print("Número de elementos de datos de entrenamiento:", data_info.splits["train"].num_examples)
print("Numero de elementos de datos de testeo:", data_info.splits["test"].num_examples)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Número de elementos de datos de entrenamiento: 60000
Numero de elementos de datos de testeo: 10000


Ahora vamos a preprocesar los datos. Para esto primero sacamos las imagenes y las etiquetas de la base de datos, y las etiquetas las transformamos en variables binarias. Después, como las imagenes son en dos dimensiones las aplanamos para que quede en una sola dimensión, y además dividimos en 255 para que los pixeles vayan entre 0 y 1. Finalmente imprimimos las dimensiones y el tipo de datos para verificar que todo se haya realizado correctamente

In [4]:
labels_true = [0, 2, 4, 6, 8]
labels_false = [1, 3, 5, 7, 9]

train_dataset_images = [example['image'] for example in tfds.as_numpy(mnist_data_train)]
train_dataset_labels = [example['label'] in labels_true for example in tfds.as_numpy(mnist_data_train)]

test_dataset_images = [example['image'] for example in tfds.as_numpy(mnist_data_test)]
test_dataset_labels = [example['label'] in labels_true for example in tfds.as_numpy(mnist_data_test)]

train_dataset_images_vector = [image.flatten()/255 for image in train_dataset_images]
test_dataset_images_vector = [image.flatten()/255 for image in test_dataset_images]

train_features = np.vstack(train_dataset_images_vector).T
train_labels = np.vstack(train_dataset_labels).T

test_features = np.vstack(test_dataset_images_vector).T
test_labels = np.vstack(test_dataset_labels).T

dim, n_train = train_features.shape
print("Los", n_train, "datos de entrenamiento tienen dimensión", dim, "y dtype", train_features.dtype)

dim, n_test = test_features.shape
print("Los", n_test, "datos de entrenamiento tienen dimensión", dim, "y dtype", test_features.dtype)

Los 60000 datos de entrenamiento tienen dimensión 784 y dtype float64
Los 10000 datos de entrenamiento tienen dimensión 784 y dtype float64


Ahora vamos a agregar de inmediato los datos necesarios para que las bases de datos sean múltiplo del tamaño local. Lo hacemos de inmediato para que las comparaciones sean justas (me hace ruido que las benchmarks tengan distinta cantidad de datos). Para esto fijo que el tamaño local sea de 64 y calculo los tamaños totales para cada uno de los conjuntos de datos, esto lo hacemos buscando el primer múltiplo de 64 que sea mayor o igual que los datos que tenemos, con un código estándar para hacer esto

In [5]:
local_size = 64
global_size_train = ((n_train + local_size - 1)//local_size)*local_size
global_size_test = ((n_test + local_size - 1)//local_size)*local_size
n_workgroups_train = global_size_train//local_size
n_workgroups_test = global_size_test//local_size

Para poder hacer el padding lo que decidí fue copiar algunos datos al azar para poder completar los que faltan. Lo hacemos a continuación y vemos los nuevos tamaños de los conjuntos de datos

In [6]:
random_indices_train = np.random.choice(n_train, global_size_train - n_train, replace=True)
random_indices_test = np.random.choice(n_test, global_size_test - n_test, replace=True)

train_features = np.hstack([train_features, train_features[:, random_indices_train]])
train_labels = np.hstack([train_labels, train_labels[:, random_indices_train]])

test_features = np.hstack([test_features, test_features[:, random_indices_test]])
test_labels = np.hstack([test_labels, test_labels[:, random_indices_test]])

dim, n_train = train_features.shape
print("Los", n_train, "datos de entrenamiento tienen dimensión", dim, "y dtype", train_features.dtype)

dim, n_test = test_features.shape
print("Los", n_test, "datos de entrenamiento tienen dimensión", dim, "y dtype", test_features.dtype)

Los 60032 datos de entrenamiento tienen dimensión 784 y dtype float64
Los 10048 datos de entrenamiento tienen dimensión 784 y dtype float64


Podemos ver que ahora si son múltiplos de 64. A continuación vemos la implementación de la regresión logística en numpy, esta fue encontrada en el material adjunto en la tarea.

In [7]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def forward_propagation(x, w, b):
    z = w.T @ x + b
    a = sigmoid(z)
    return a

def backward_propagation(x, a, y):
    m = x.shape[1]
    j = -np.sum(y * np.log(a) + (1-y) * np.log(1-a)) / m
    dz = a - y
    dw = (x @ dz.T) / m
    db = np.mean(dz)
    return dw, db, j

def update(w, b, dw, db, lr):
    w -= lr * dw
    b -= lr * db
    return w, b

def train(x, y, w, b, lr, n_iter):
    costs = []
    for k in range(n_iter):
        a = forward_propagation(x, w, b)
        dw, db, j = backward_propagation(x, a, y)
        w, b = update(w, b, dw, db, lr)
        costs.append(j)
    return w, b, costs

def predict(x, w, b):
    a = forward_propagation(x, w, b)
    return np.round(a)

Establecemos los parámetros del modelo

In [8]:
learning_rate = 0.01
initial_weights = np.zeros([dim, 1], dtype=float)
initial_bias = 0.0
n_iterations = 1000

Y lo entrenamos

In [9]:
time_start = time.time()
weights, bias, _ = train(train_features, train_labels, initial_weights, initial_bias, learning_rate, n_iterations)
print(f"Se tardó {time.time() - time_start:.2f} segundos en entrenar")

Se tardó 21.43 segundos en entrenar


Notamos que se demoró 21 segundos, ahora veamos si está funcionando correctamente

In [10]:
time_start = time.time()

prediction_train = predict(train_features, weights, bias)
prediction_test = predict(test_features, weights, bias)

print(f"Se tardó {time.time() - time_start:.2f} segundos en predecir")

correct_train = np.sum(prediction_train == train_labels)
correct_test = np.sum(prediction_test == test_labels)

print("Predichas", correct_train, "datos correctamente de los", n_train, "datos de entrenamiento:", correct_train/n_train*100, "%")
print("Predichas", correct_test, "datos correctamente de los", n_test, "datos de testeo:", correct_test/n_test*100, "%")

Se tardó 0.01 segundos en predecir
Predichas 51341 datos correctamente de los 60032 datos de entrenamiento: 85.52272121535182 %
Predichas 8629 datos correctamente de los 10048 datos de testeo: 85.87778662420382 %


Predijo una cantidad razonable de datos correctamente, así que concluímos que funciona correctamente. Además notamos que la predicción fue prácticamente instantanea. Ahora vamos a implementar la regresión en OpenPyCL, lo primero será ver los dispositivos que tenemos disponibles.

In [11]:
for platform in cl.get_platforms():
    print(f"Plataforma: {platform.name}")
    for device in platform.get_devices():
        print(f"    Dispositivo: {device.name}")
        print(f"    Tipo de dispositivo: {cl.device_type.to_string(device.type)}")

Plataforma: NVIDIA CUDA
    Dispositivo: NVIDIA A100-SXM4-40GB
    Tipo de dispositivo: ALL | GPU
Plataforma: Portable Computing Language
    Dispositivo: pthread-Intel(R) Xeon(R) CPU @ 2.20GHz
    Tipo de dispositivo: ALL | CPU


Vemos que tenemos una GPU y una CPU disponibles. Notemos que tengo la GPU y CPU pagadas de colab porque se me acabó el cómputo y me quedaba poco para términas así que me compré unidades de cómputo XD. Seleccionemos la GPU primero

In [12]:
platform = cl.get_platforms()[0]
device = platform.get_devices()[0]
print("Nombre plataforma:", platform.name)
print("Nombre dispositivo:", device.name)
print(f"Tipo de dispositivo: {cl.device_type.to_string(device.type)}")
print("Maximo tamaño de grupo de trabajo:", device.max_work_group_size)

Nombre plataforma: NVIDIA CUDA
Nombre dispositivo: NVIDIA A100-SXM4-40GB
Tipo de dispositivo: ALL | GPU
Maximo tamaño de grupo de trabajo: 1024


Podemos ver que la hemos seleccionado correctamente y podemos ver sus características. Ahora creemos el contexto y la cola de trabajo

In [13]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx, properties=cl.command_queue_properties.PROFILING_ENABLE)

Tal como se pide, implementé tres kernels separados para las tareas de propagación hacia adelante (`forward`), retropropagación (`backward`), y predicción (`predict`). A continuación, detallo el diseño SIMD de cada uno:

## **`forward`**
- **Descripción del diseño SIMD**:  
  Cada hilo procesa un dato en paralelo, realizando las siguientes operaciones:
  1. Calcula el producto punto entre el vector de pesos $W$ y una instancia de los datos de entrada $X$.
  2. Suma el sesgo $b$ al resultado del producto punto.
  3. Aplica la función sigmoide $\sigma(z) = \frac{1}{1 + e^{-z}}$ para obtener la salida activada.

- **Paralelismo**:  
  El procesamiento paralelo se asegura asignando a cada hilo la tarea de procesar una instancia específica del conjunto de datos $X$. Esto permite realizar el cálculo de manera independiente para cada dato.


## **`backward`**
- **Descripción del diseño SIMD**:  
  Este kernel se encarga de calcular los gradientes parciales para el sesgo $b$ y los pesos $W$. Para cada grupo de hilos se hace lo siguiente:
  1. Un hilo representante acumula las contribuciones de los gradientes generadas por los demás hilos del grupo.
  2. Los cálculos incluyen:
     - La diferencia $dz = a - y$, lo cual es la contribución al gradiente de $b$
     - La contribución del gradiente de $W$ como un producto punto entre $dz$ y las características de entrada $X$.
  3. Finalmente, la suma acumulada de los gradientes del grupo se combina en la CPU mediante una reducción global (tal como se vio en ayudantía)

- **Consideraciones de implementación**:  
  - El paralelismo se da entre los grupos, ya que los grupos pueden realizar sus operaciones en paralelo (tal como se vio en los códigos entregados en ayudantía)
  - La acumulación de gradientes dentro de un grupo se realiza utilizando un ciclo `for` en lugar de una suma en forma de árbol binario. Esto se debe a que en la GPU estándar esta aproximación resultó más rápida. En el caso de la GPU de pago ambas aproximaciones tienen tiempos similares.  
  - La implementación de suma en forma de árbol binario está disponible al final del notebook.


## **`predict`**
- **Descripción del diseño SIMD**:  
  Cada hilo realiza predicciones de manera independiente para una instancia de entrada, llevando a cabo las siguientes operaciones:
  1. Calcula el producto punto entre $W$ y la instancia correspondiente de $X$.
  2. Suma el término de sesgo $b$.
  3. Aplica una función de umbral para determinar la clase predicha: $ y_{\text{pred}} = 1.0 $ si $ z > 0 $, de lo contrario $ y_{\text{pred}} = 0.0 $.

- **Paralelismo**:  
  Similar al kernel de propagación hacia adelante, cada hilo procesa una instancia diferente de manera independiente, garantizando el aprovechamiento del modelo SIMD.


In [14]:
kernel = """
__kernel void forward(
    __global const double *W,
    __global const double *X,
    const double b,
    __global double *a,
    const int d)
{
    int global_id = get_global_id(0);

    double z = b;
    int offset = global_id*d;
    for(int i = 0; i<d; i++){
        z += W[i]*X[offset + i];
    }

    a[global_id] = 1.0/(1.0 + exp(-z));
}

__kernel void backward(
    __global const double *a,
    __global const double *X,
    __global const double *y,
    __global double *partial_b,
    __global double *partial_w,
    const int d,
    const int m,
    const int tot_groups)
{
    int group_size = get_local_size(0);
    int local_id = get_local_id(0);
    int group_id = get_group_id(0);
    int global_id = get_global_id(0);

    if(local_id == 0){
        partial_b[group_id] = 0;
        for(int j = 0; j<d; j++){
            partial_w[j*tot_groups + group_id] = 0;
        }
        for(int i = 0; i<group_size; i++){
            double dz = a[global_id + i] - y[global_id + i];
            partial_b[group_id] += dz;
            for(int j = 0; j<d; j++){
                partial_w[j*tot_groups + group_id] += dz*X[(global_id + i)*d + j];
            }
        }
    }
}

__kernel void predict(
    __global const double *W,
    __global const double *X,
    const double b,
    __global double *y_pred,
    const int d)
{
    int global_id = get_global_id(0);

    double z = b;
    int offset = global_id*d;
    for(int i = 0; i<d; i++){
        z += W[i]*X[offset + i];
    }

    y_pred[global_id] = (z > 0) * 1.0;
}
"""

Compilamos

In [15]:
prg = cl.Program(ctx, kernel).build()

Vemos que ha compilado correctamente, así que creamos los arreglos necesarios

In [16]:
db_train_labels = train_labels.astype(np.float64).squeeze()

cl_Xtrain = cl_array.to_device(queue, train_features.T)
cl_ytrain = cl_array.to_device(queue, db_train_labels)
cl_Xtest = cl_array.to_device(queue, test_features.T)

cl_W = cl_array.empty(queue, (dim,), dtype=np.float64)
cl_b = cl_array.empty(queue, (1,), dtype=np.float64)

cl_Wgrad = cl_array.empty(queue, (dim*n_workgroups_train,), dtype=np.float64)
cl_bgrad = cl_array.empty(queue, (n_workgroups_train,), dtype=np.float64)
cl_Wgradpartial = cl_array.empty(queue, (dim*n_workgroups_train,), dtype=np.float64)
cl_bgradpartial = cl_array.empty(queue, (n_workgroups_train,), dtype=np.float64)
cl_a = cl_array.empty(queue, (n_train,), dtype=np.float64)

cl_predtrain = cl_array.empty(queue, (n_train,), dtype=np.float64)
cl_predtest = cl_array.empty(queue, (n_test,), dtype=np.float64)

Y finalmente entrenamos el modelo usando PyOpenCL en la GPU

In [17]:
time_start = time.time()
time_kernel = 0

for _ in range(n_iterations):

    event = prg.forward(queue, (global_size_train,), (local_size, ), cl_W.data, cl_Xtrain.data, cl_b.data, cl_a.data, np.int32(dim))
    event.wait()
    time_kernel += 1e-9 *(event.profile.end - event.profile.start)
    event = prg.backward(queue, (global_size_train,), (local_size, ), cl_a.data, cl_Xtrain.data, cl_ytrain.data, cl_bgrad.data, cl_Wgrad.data, np.int32(dim), np.int32(n_train), np.int32(n_workgroups_train))
    event.wait()
    time_kernel += 1e-9 *(event.profile.end - event.profile.start)

    bgrad = float(sum(cl_bgrad.get()))
    Wgrad = np.add.reduceat(cl_Wgrad.get(), np.arange(0, dim*n_workgroups_train, n_workgroups_train))

    cl_b.set(cl_b.get() - (learning_rate/n_train) * bgrad)
    cl_W.set(cl_W.get() - (learning_rate/n_train) * Wgrad)

print(f"Se tardó {time.time() - time_start:.2f} segundos en entrenar, de los cuales {time_kernel:.2f} fueron en el kernel")


Se tardó 20.50 segundos en entrenar, de los cuales 17.39 fueron en el kernel


Vemos que se demoró parecido a la implementación en numpy, y la mayor parte fueron cómputos en el kernel. En el entorno gratis esta se demoraba como la mitad de la implementación en numpy (esta como 45s y la otra como 80s), pero ahora parece que anduvieron igualados. Hagamos las predicciones y veamos el error

In [18]:
time_start = time.time()
time_kernel = 0
event = prg.predict(queue, (global_size_train,), (local_size,), cl_W.data, cl_Xtrain.data, cl_b.data, cl_predtrain.data, np.int32(dim))
event.wait()
time_kernel += 1e-9 *(event.profile.end - event.profile.start)
predtrain = cl_predtrain.get()
event = prg.predict(queue, (global_size_test,), (local_size,), cl_W.data, cl_Xtest.data, cl_b.data, cl_predtest.data, np.int32(dim))
event.wait()
time_kernel += 1e-9 *(event.profile.end - event.profile.start)
predtest = cl_predtest.get()
print(f"Se tardó {time.time() - time_start:.2f} segundos en predecir, de los cuales {time_kernel:.2f} fueron en el kernel")

correct_train = np.sum(predtrain == train_labels)
correct_test = np.sum(predtest == test_labels)

print("Predichas", correct_train, "datos correctamente de los", n_train, "datos de entrenamiento:", correct_train/n_train*100, "%")
print("Predichas", correct_test, "datos correctamente de los", n_test, "datos de testeo:", correct_test/n_test*100, "%")

Se tardó 0.00 segundos en predecir, de los cuales 0.00 fueron en el kernel
Predichas 51253 datos correctamente de los 60032 datos de entrenamiento: 85.37613272921108 %
Predichas 8610 datos correctamente de los 10048 datos de testeo: 85.68869426751591 %


Notamos que obtenemos una buena precisión por lo que el modelo está funcionando correctamente. Además observamos que las predicciones fueron prácticamente instantaneas. Ahora veamos como anda en la CPU, lo primero es que vamos a crear el entorno, lo haremos todo en este bloque de código ya que es clásico a estas alturas (y el kernel obviamente será el mismo)

In [19]:
platform = cl.get_platforms()[1]
device = platform.get_devices()[0]
print("Nombre plataforma:", platform.name)
print("Nombre dispositivo:", device.name)
print(f"Tipo de dispositivo: {cl.device_type.to_string(device.type)}")
print("Maximo tamaño de grupo de trabajo:", device.max_work_group_size)
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx, properties=cl.command_queue_properties.PROFILING_ENABLE)
prg = cl.Program(ctx, kernel).build()

Nombre plataforma: Portable Computing Language
Nombre dispositivo: pthread-Intel(R) Xeon(R) CPU @ 2.20GHz
Tipo de dispositivo: ALL | CPU
Maximo tamaño de grupo de trabajo: 4096


Notamos que se escogió correctamente la CPU, ahora creamos los arreglos que usaremos

In [20]:
db_train_labels = train_labels.astype(np.float64).squeeze()

cl_Xtrain = cl_array.to_device(queue, train_features.T)
cl_ytrain = cl_array.to_device(queue, db_train_labels)
cl_Xtest = cl_array.to_device(queue, test_features.T)

cl_W = cl_array.empty(queue, (dim,), dtype=np.float64)
cl_b = cl_array.empty(queue, (1,), dtype=np.float64)

cl_Wgrad = cl_array.empty(queue, (dim*n_workgroups_train,), dtype=np.float64)
cl_bgrad = cl_array.empty(queue, (n_workgroups_train,), dtype=np.float64)
cl_Wgradpartial = cl_array.empty(queue, (dim*n_workgroups_train,), dtype=np.float64)
cl_bgradpartial = cl_array.empty(queue, (n_workgroups_train,), dtype=np.float64)
cl_a = cl_array.empty(queue, (n_train,), dtype=np.float64)

cl_predtrain = cl_array.empty(queue, (n_train,), dtype=np.float64)
cl_predtest = cl_array.empty(queue, (n_test,), dtype=np.float64)

Y ahora entrenamos el modelo en la CPU

In [21]:
time_start = time.time()
time_kernel = 0

for _ in range(n_iterations):

    event = prg.forward(queue, (global_size_train,), (local_size, ), cl_W.data, cl_Xtrain.data, cl_b.data, cl_a.data, np.int32(dim))
    event.wait()
    time_kernel += 1e-9 *(event.profile.end - event.profile.start)
    event = prg.backward(queue, (global_size_train,), (local_size, ), cl_a.data, cl_Xtrain.data, cl_ytrain.data, cl_bgrad.data, cl_Wgrad.data, np.int32(dim), np.int32(n_train), np.int32(n_workgroups_train))
    event.wait()
    time_kernel += 1e-9 *(event.profile.end - event.profile.start)

    bgrad = float(sum(cl_bgrad.get()))
    Wgrad = np.add.reduceat(cl_Wgrad.get(), np.arange(0, dim*n_workgroups_train, n_workgroups_train))

    cl_b.set(cl_b.get() - (learning_rate/n_train) * bgrad)
    cl_W.set(cl_W.get() - (learning_rate/n_train) * Wgrad)

print(f"Se tardó {time.time() - time_start:.2f} segundos en entrenar, de los cuales {time_kernel:.2f} fueron en el kernel")


Se tardó 20.50 segundos en entrenar, de los cuales 17.40 fueron en el kernel


Notamos que se demoró similar a la GPU y a numpy, se siente un poco raro ya que en el entorno gratis andaba mejor que numpy y un poco peor que la GPU. Además notamos que la mayor parte del tiempo fue trabajo en el kernel. Ahora veamos la precisión del modelo

In [22]:
time_start = time.time()
time_kernel = 0
event = prg.predict(queue, (global_size_train,), (local_size,), cl_W.data, cl_Xtrain.data, cl_b.data, cl_predtrain.data, np.int32(dim))
event.wait()
time_kernel += 1e-9 *(event.profile.end - event.profile.start)
predtrain = cl_predtrain.get()
event = prg.predict(queue, (global_size_test,), (local_size,), cl_W.data, cl_Xtest.data, cl_b.data, cl_predtest.data, np.int32(dim))
event.wait()
time_kernel += 1e-9 *(event.profile.end - event.profile.start)
predtest = cl_predtest.get()
print(f"Se tardó {time.time() - time_start:.2f} segundos en predecir, de los cuales {time_kernel:.2f} fueron en el kernel")

correct_train = np.sum(predtrain == train_labels)
correct_test = np.sum(predtest == test_labels)

print("Predichas", correct_train, "datos correctamente de los", n_train, "datos de entrenamiento:", correct_train/n_train*100, "%")
print("Predichas", correct_test, "datos correctamente de los", n_test, "datos de testeo:", correct_test/n_test*100, "%")

Se tardó 0.00 segundos en predecir, de los cuales 0.00 fueron en el kernel
Predichas 51253 datos correctamente de los 60032 datos de entrenamiento: 85.37613272921108 %
Predichas 8610 datos correctamente de los 10048 datos de testeo: 85.68869426751591 %


Notamos que obtuvimos la misma precisión que antes, por lo que el modelo funcionó correctamente. Además nuevamente la predicción fue casi instantanea

Por último, dejo la implementación del backward en donde se hace la reducción en forma de árbol binario. En el entorno gratis esto se demoraba como el doble, pero en el entorno de pago esto se demoró prácticamente lo mismo

In [23]:
platform = cl.get_platforms()[0]
device = platform.get_devices()[0]

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx, properties=cl.command_queue_properties.PROFILING_ENABLE)

kernel = """
__kernel void forward(
    __global const double *W,
    __global const double *X,
    const double b,
    __global double *a,
    const int d)
{
    int global_id = get_global_id(0);

    double z = b;
    int offset = global_id*d;
    for(int i = 0; i<d; i++){
        z += W[i]*X[offset + i];
    }

    a[global_id] = 1.0/(1.0 + exp(-z));
}

__kernel void backward(
    __global const double *a,
    __global const double *X,
    __global const double *y,
    __global double *grad_b,
    __global double *grad_w,
    __global double *grad_bpartial,
    __global double *grad_wpartial,
    const int d,
    const int m,
    const int tot_groups)
{
    int group_size = get_local_size(0);
    int local_id = get_local_id(0);
    int group_id = get_group_id(0);
    int global_id = get_global_id(0);

    double dz = a[global_id] - y[global_id];
    int offset = global_id*d;

    grad_b[global_id] = dz;
    for(int j = 0; j<d; j++){
        grad_w[offset + j] += dz*X[offset + j];
    }

    int step = 2;
    while(step <= group_size){
        if(local_id%step == 0){
            grad_b[global_id] += grad_b[global_id + step/2];
            for(int j = 0; j<d; j++){
                grad_w[offset + j] += dz*X[(global_id + step/2)*d + j];
            }
        }
        barrier(CLK_GLOBAL_MEM_FENCE);
        step *= 2;
    }

    if(local_id == 0){
        grad_bpartial[group_id] = grad_b[global_id];
        for(int j = 0; j<d; j++){
            grad_wpartial[j*tot_groups + group_id] = grad_w[offset + j];
        }
    }
}

__kernel void predict(
    __global const double *W,
    __global const double *X,
    const double b,
    __global double *y_pred,
    const int d)
{
    int global_id = get_global_id(0);

    double z = b;
    int offset = global_id*d;
    for(int i = 0; i<d; i++){
        z += W[i]*X[offset + i];
    }

    y_pred[global_id] = (z > 0) * 1.0;
}
"""

prg = cl.Program(ctx, kernel).build()

db_train_labels = train_labels.astype(np.float64).squeeze()

cl_Xtrain = cl_array.to_device(queue, train_features.T)
cl_ytrain = cl_array.to_device(queue, db_train_labels)
cl_Xtest = cl_array.to_device(queue, test_features.T)

cl_W = cl_array.empty(queue, (dim,), dtype=np.float64)
cl_b = cl_array.empty(queue, (1,), dtype=np.float64)

cl_Wgrad = cl_array.empty(queue, (dim*n_train,), dtype=np.float64)
cl_bgrad = cl_array.empty(queue, (n_train,), dtype=np.float64)
cl_Wgradpartial = cl_array.empty(queue, (dim*n_workgroups_train,), dtype=np.float64)
cl_bgradpartial = cl_array.empty(queue, (n_workgroups_train,), dtype=np.float64)
cl_a = cl_array.empty(queue, (n_train,), dtype=np.float64)

cl_predtrain = cl_array.empty(queue, (n_train,), dtype=np.float64)
cl_predtest = cl_array.empty(queue, (n_test,), dtype=np.float64)

time_start = time.time()

for _ in range(n_iterations):

    prg.forward(queue, (global_size_train,), (local_size, ), cl_W.data, cl_Xtrain.data, cl_b.data, cl_a.data, np.int32(dim))
    prg.backward(queue, (global_size_train,), (local_size, ), cl_a.data, cl_Xtrain.data, cl_ytrain.data, cl_bgrad.data, cl_Wgrad.data, cl_bgradpartial.data, cl_Wgradpartial.data, np.int32(dim), np.int32(n_train), np.int32(n_workgroups_train))

    bgrad = float(sum(cl_bgradpartial.get()))
    Wgrad = np.add.reduceat(cl_Wgradpartial.get(), np.arange(0, dim*n_workgroups_train, n_workgroups_train))
    cl_b.set(cl_b.get() - (learning_rate/n_train) * bgrad)
    cl_W.set(cl_W.get() - (learning_rate/n_train) * Wgrad)

prg.predict(queue, (global_size_train,), (local_size,), cl_W.data, cl_Xtrain.data, cl_b.data, cl_predtrain.data, np.int32(dim))
predtrain = cl_predtrain.get()
prg.predict(queue, (global_size_test,), (local_size,), cl_W.data, cl_Xtest.data, cl_b.data, cl_predtest.data, np.int32(dim))
predtest = cl_predtest.get()

print(f"Se tardó {time.time() - time_start:.2f} segundos")

Se tardó 20.53 segundos


Vemos que se demoró unos 20 segundos tal como los anteriores